In [0]:
Create or replace temporary view gas as (
WITH prepared AS (
  SELECT 
    TradingDate,
    -- Replace 0 with NULL so we can apply last_value over non-zero values
    CASE 
      WHEN GasVWAPDollarsPerGigajoule = 0 THEN NULL 
      ELSE GasVWAPDollarsPerGigajoule 
    END AS NonZeroValue
  FROM datahub.wholesale.gasandcarbonpriceindices
  WHERE TradingDate BETWEEN '2019-01-01' AND '2025-06-30'
),
with_filled AS (
  SELECT 
    TradingDate,
    -- Use last non-null (i.e., non-zero) value
    LAST_VALUE(NonZeroValue, TRUE) OVER (
      ORDER BY TradingDate 
      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS GasVWAPDollarsPerGigajoule
  FROM prepared
), Daily_price as (
select TradingDate, Avg(DollarsPerMegawattHour) as OTA_price from ea_prd.gold.factpredispatchandfinalenergyprices
where TradingDate BETWEEN '2020-07-01' AND '2025-06-30'
and PointOfConnectionCode = 'OTA2201' and PriceTypeCode = 'F'
group by TradingDate
)
SELECT CASE
    WHEN MONTH(a.TradingDate) <= 6 THEN (YEAR(a.TradingDate) - 1) || '/' || YEAR(a.TradingDate)
    ELSE YEAR(a.TradingDate) || '/' || (YEAR(a.TradingDate) + 1)
  END AS `Year`, 
  a.TradingDate, 
  a.GasVWAPDollarsPerGigajoule, 
  AVG(GasVWAPDollarsPerGigajoule) OVER (
      ORDER BY a.TradingDate 
      ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
    ) AS RollingAvg30DollarsPerGJ,
  a.GasVWAPDollarsPerGigajoule*10 as Scaled_Gas_Price, 
  b.OTA_price 
FROM with_filled a INNER JOIN Daily_price b ON a.TradingDate = b.TradingDate
WHERE a.TradingDate BETWEEN '2021-07-01' AND '2025-06-30'
order by a.TradingDate
);

select * from gas

Databricks visualization. Run in Databricks to view.

In [0]:
Create or replace temporary view carbon as (
WITH prepared AS (
  SELECT 
    TradingDate,
    -- Replace 0 with NULL so we can apply last_value over non-zero values
    CASE 
      WHEN CarbonVWAPDollarsPerNZU = 0 THEN NULL 
      ELSE CarbonVWAPDollarsPerNZU 
    END AS NonZeroValue
  FROM datahub.wholesale.gasandcarbonpriceindices
  WHERE TradingDate BETWEEN '2019-01-01' AND '2025-06-30'
),
with_filled AS (
  SELECT 
    TradingDate,
    -- Use last non-null (i.e., non-zero) value
    LAST_VALUE(NonZeroValue, TRUE) OVER (
      ORDER BY TradingDate 
      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS CarbonVWAPDollarsPerNZU
  FROM prepared
)
SELECT 
  CASE
    WHEN MONTH(TradingDate) <= 6 THEN (YEAR(TradingDate) - 1) || '/' || YEAR(TradingDate)
    ELSE YEAR(TradingDate) || '/' || (YEAR(TradingDate) + 1)
  END AS `Year`, 
  TradingDate, 
  CarbonVWAPDollarsPerNZU,
  AVG(CarbonVWAPDollarsPerNZU) OVER (
      ORDER BY TradingDate 
      ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
    ) AS RollingAvg30DollarsPerNZU
FROM with_filled
WHERE TradingDate BETWEEN '2021-07-01' AND '2025-06-30'
);

SELECT * from carbon

In [0]:
select gas.TradingDate, gas.GasVWAPDollarsPerGigajoule as Daily_ETS_inclusive_Gas_Price, 
gas.RollingAvg30DollarsPerGJ as 30daysRollingAverage_ETS_inclusive_Gas_Price, 
gas.RollingAvg30DollarsPerGJ - carbon.RollingAvg30DollarsPerNZU* 0.05397 as 30daysRollingAverage_ETS_exnclusive_Gas_Price
from carbon inner join gas on carbon.TradingDate = gas.TradingDate